In [1]:
# This R environment comes with all of CRAN preinstalled, as well as many other helpful packages
# The environment is defined by the kaggle/rstats docker image: https://github.com/kaggle/docker-rstats
# For example, here's several helpful packages to load in 

library(ggplot2) # Data visualization
library(readr) # CSV file I/O, e.g. the read_csv function

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

system("ls ../input")

# Any results you write to the current directory are saved as output.

In [ ]:
dataset <-as.data.frame(read.csv('../input/Data v2.csv'))
colnames(dataset)[colnames(dataset)=="default.payment.next.month"] <- "Defaulted"

In [ ]:
all_cols = names(dataset)

categorical_columns <- c('Sex','Education','Marriage')

for (c in categorical_columns){
    temp_table <- table(dataset[,'Defaulted'], dataset[,c])
    barplot(temp_table, col=c("red", "blue"), legend=rownames(temp_table),main = 'Defaul Rates',xlab = c,ylab = '# Records')
    }



In [ ]:
aggregate(dataset[, 'Defaulted'], list(dataset$Sex), mean);
aggregate(dataset[, 'Defaulted'], list(dataset$Education), mean);
aggregate(dataset[, 'Defaulted'], list(dataset$Marriage), mean)
aggregate(dataset[, 'Defaulted'], list(dataset$AGE), mean)
mean(dataset$Defaulted)

In [ ]:
library(corrplot)
library(ggcorrplot)

numeric_columns<-c('April_Pay', 'May_Pay', 'June_Pay', 'July_Pay', 'August_Pay' ,'September_Pay', 'April_Bill_Amt', 'May_Bill_Amt' ,'June_Bill_Amt',
                   'July_Bill_Amt', 'August_Bill_Amt', 'September_Bill_Amt' ,'April_Pay_Amt', 'May_Pay_Amt', 'June_Pay_Amt', 'July_Pay_Amt', 'August_Pay_Amt', 'September_Pay_Amt','Defaulted')

cor_ind = round(cor(dataset[numeric_columns]),1)
p.mat <- cor_pmat(dataset[numeric_columns])


ggcorrplot(cor_ind, p.mat = p.mat, hc.order = FALSE,
           , insig = "blank", lab = TRUE,lab_size =2) +
theme(axis.text.x=element_text(size=7, angle=45, vjust=1, hjust=1, 
                               margin=margin(-3,0,0,0)),
      axis.text.y=element_text(size=7, margin=margin(0,-3,0,0)),
      panel.grid.major=element_blank(),
      plot.title = element_text( face="bold", size=10, hjust=0)) +
  ggtitle("Correlation between various numeric columns")

In [ ]:
for (c in c(categorical_columns) ){
dataset[,c]<- factor(dataset[,c])
    }

In [ ]:
## 75% of the sample size
smp_size <- floor(0.75 * nrow(dataset))

## set the seed to make your partition reproductible
set.seed(123)
train_ind <- sample(seq_len(nrow(dataset)), size = smp_size)

train <- dataset[train_ind, ]
test <- dataset[-train_ind, ]

train_x <-data.frame(train[,-c(1,25)])
test_x <-data.frame(test[,-c(1,25)])
train_y<-train[,25]
test_y<-test[,25]

In [ ]:
require(xgboost)
set.seed(100)

x_train <- model.matrix(train_y~., train_x)[,-1]
x_test <- model.matrix(test_y~., test_x)[,-1]
xgb.mod = xgboost(data = as.matrix(x_train), nfold = 5, label = as.matrix(train_y), 
    nrounds = 2000, verbose = FALSE, objective = "binary:logistic", eval_metric = "error", 
    nthread = 8, eta = 0.001, gamma = 0.0468, max_depth = 10, min_child_weight = 1.7817, 
    subsample = 0.5213, colsample_bytree = 0.4603)
xgb.importance(colnames(as.matrix(x_train)), model = xgb.mod)

## Predictions
xgb.pred_train <- predict(xgb.mod, newdata = as.matrix(x_train))
xgb.pred_test <- predict(xgb.mod, newdata = as.matrix(x_test))


In [ ]:
library(pROC)
auc(test_y,xgb.pred_test)

In [ ]:
#xgb.pred_test <- as.numeric(xgb.pred_test>0.9)
library(caret)
#confusionMatrix(test_y,as.numeric(xgb.pred_test>0.5))
xgb.pred_test_outcome <- (xgb.pred_test - min(xgb.pred_test))/(max(xgb.pred_test)-min(xgb.pred_test))
confusionMatrix(test_y,as.numeric(xgb.pred_test_outcome>0.3))

In [ ]:
model <- glm (Defaulted ~ ., data = train[-1], family = binomial)
predict <- predict(model,test_x)
predict_outcome <- (predict - min(predict))/(max(predict)-min(predict))
auc(test_y,predict_outcome)
confusionMatrix(test_y,as.numeric(predict_outcome>0.2))